In [ ]:
!mkdir Dataset
!tar --extract --file /kaggle/input/mapbox/DataSet.tgz
!mv Mapbox Dataset

In [ ]:
!pip install -U wandb 
!pip install -U ipywidgets
!pip install ipysheet
!pip install pytorch-lightning
!pip install lightning
import wandb
import os
import sys
import yaml
wandb.login(key="Ваш ключ WandB", relogin=True)
run = wandb.init(project="Upscaler", name = "Upscaler Base")

In [ ]:
import math

import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from PIL import Image
from torchvision import transforms
from torchvision.utils import save_image
import gc

class Upscaler(nn.Module):

    def __init__(self, input_size):
        super(Upscaler, self).__init__()

        # Encoder
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)

        # Progressive Decoder
        self.upsample1 = nn.UpsamplingNearest2d(scale_factor=2.0)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.upsample2 = nn.UpsamplingNearest2d(scale_factor=2.0)
        self.conv4 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.conv6 = nn.Conv2d(64, 3, kernel_size=3, padding=1)

        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.pool(x)

        x = self.upsample1(x)
        x = self.relu(self.conv3(x))
        x = self.upsample2(x)
        x = self.relu(self.conv4(x))
        x = self.conv6(x)
        return x


    def train_model(model, train_loader, val_loader, num_epochs, device):

        # Move the model to the device
        model.load_state_dict(torch.load("/kaggle/working/modelDict.pt"))
        model.to(device)

        # Define the loss function and optimizer
        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=0.001)

        # Training loop
        for epoch in range(num_epochs):
            # Training
            model.train()
            train_loss = 0.0
            for high_res, trash in train_loader:
                print(high_res.size())

                low_res = model.getLowRes(high_res)

                high_res = high_res.to(device)
                low_res = low_res.to(device)

                # Forward pass
                output = model(low_res)
                loss = criterion(output, high_res)

                # Backward pass and optimization
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                train_loss += loss.item()
                gc.collect()
                torch.cuda.empty_cache()

            train_loss /= len(train_loader)

            # Validation
            model.eval()
            val_loss = 0.0
            with torch.no_grad():
                for high_res, trash in val_loader:
                    low_res = model.getLowRes(high_res)

                    high_res = high_res.to(device)
                    low_res = low_res.to(device)

                    output = model(low_res)
                    val_loss += criterion(output, high_res).item()
                    gc.collect()
                    torch.cuda.empty_cache()

            val_loss /= len(val_loader)
            # Print the progress
            print(f'Epoch [{epoch + 1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')

            # Save a sample output image
            torch.save(model.state_dict(), "modelDict.pt")
            sample_output = model(low_res[:1])
            save_image(sample_output, f'sample_output_epoch_{epoch + 1}.png')

        return model

    def getLowRes(self, high_res):
        img_array = high_res.numpy()
        img_array = np.transpose(img_array, (0, 2, 3, 1))
        resized_images = []
        for img in img_array:
            resized_img = cv2.resize(img, (400, 400))  # Resize to (64, 64)
            resized_images.append(resized_img)
        resized_images = np.array(resized_images)
        resized_tensor = torch.from_numpy(resized_images)
        # Transpose back to tensor shape (32, 3, 64, 64)
        return resized_tensor.permute(0, 3, 1, 2)

    def UpcsalerInference(self):
        model = Upscaler()
        model.load_state_dict(torch.load('upscaler_model.pth'))
        model.eval()

        # Define the preprocessing transformations
        preprocess = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

        # Open the input low-resolution image
        input_image = Image.open('input_image.jpg')

        # Preprocess the input image
        input_tensor = preprocess(input_image).unsqueeze(0)

        # Upscale the input image
        with torch.no_grad():
            output_tensor = model(input_tensor)

        # Postprocess the output tensor
        output_tensor = output_tensor.squeeze(0)
        output_tensor = output_tensor.permute(1, 2, 0)  # CHW to HWC
        output_image = transforms.ToPILImage()(output_tensor)

        # Save the upscaled image
        output_image.save('upscaled_image.jpg')


In [ ]:
gc.collect()
torch.cuda.empty_cache()

In [ ]:
import torch.utils.data


from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader
gc.collect()
torch.cuda.empty_cache()
# Example usage
high_res_folder = 'Dataset'
transform = transforms.Compose([
    transforms.Resize((800, 800)),
    transforms.ToTensor()
])

high_res_dataset = ImageFolder(high_res_folder, transform=transform)

generator1 = torch.Generator().manual_seed(21)

train_set, val_set, other = torch.utils.data.random_split(high_res_dataset, [0.5, 0.1, 0.4], generator1)
train_loader = DataLoader(train_set, batch_size = 8, shuffle=True, )
val_loader = DataLoader(val_set, batch_size=8, shuffle=True,)

del high_res_dataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = Upscaler((400,400))
model = nn.DataParallel(model)
model.module.train_model( train_loader=train_loader, val_loader=val_loader, num_epochs=5,
                  device=device)


# PL

In [8]:
# Путь к папке с изображениями
image_folder = '/kaggle/working/Dataset/Mapbox'

# Пороговое значение для удаления изображений
threshold = 20
for filename in os.listdir(image_folder):
    img_path = os.path.join(image_folder, filename)
    
    # Проверяем, является ли файл изображением
    if img_path.endswith(('.jpg', '.png', '.bmp')):
        
        # Загружаем изображение
        img = cv2.imread(img_path)
        
        # Вычисляем среднее значение по каналам цвета
        mean = cv2.mean(img)
        
        # Если все средние значения близки друг к другу (в пределах порогового значения),
        # то изображение считается однотонным
        if abs(mean[0] - mean[1]) < threshold and abs(mean[1] - mean[2]) < threshold and abs(mean[0] - mean[2]) < threshold:
            # Удаляем изображение
            os.remove(img_path)
            print(f'Удалено изображение: {filename}')

Удалено изображение: 5013.png
Удалено изображение: 2284.png
Удалено изображение: 5283.png
Удалено изображение: 9940.png
Удалено изображение: 4048.png
Удалено изображение: 3066.png
Удалено изображение: 9689.png
Удалено изображение: 3965.png
Удалено изображение: 2104.png
Удалено изображение: 4572.png
Удалено изображение: 3251.png
Удалено изображение: 9303.png
Удалено изображение: 7087.png
Удалено изображение: 7440.png
Удалено изображение: 6071.png
Удалено изображение: 9781.png
Удалено изображение: 9728.png
Удалено изображение: 3354.png
Удалено изображение: 5186.png
Удалено изображение: 1298.png
Удалено изображение: 55.png
Удалено изображение: 9469.png
Удалено изображение: 1620.png
Удалено изображение: 2193.png
Удалено изображение: 3541.png
Удалено изображение: 2081.png
Удалено изображение: 5156.png
Удалено изображение: 1678.png
Удалено изображение: 9795.png
Удалено изображение: 9123.png
Удалено изображение: 7612.png
Удалено изображение: 8716.png
Удалено изображение: 3315.png
Удалено изоб

In [1]:
import wandb
import os
import sys
import yaml
wandb.login(key="504d320dca3edf2d54e8a21e0c24f4251cd7e0c0", relogin=True)
run = wandb.init(project="Upscaler", name = "Upscaler Base")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: kamenevv2. Use `wandb login --relogin` to force relogin


In [24]:
import math
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from PIL import Image
from torchvision import transforms
from torchvision.utils import save_image
import pytorch_lightning as pl
import gc

from torchmetrics.image import StructuralSimilarityIndexMeasure
from torchmetrics.image import PeakSignalNoiseRatio

class Upscaler(pl.LightningModule):

    def __init__(self, input_size):
        super(Upscaler, self).__init__()

        # Encoder
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)

        # Progressive Decoder
        self.upsample1 = nn.UpsamplingNearest2d(scale_factor=2.0)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.upsample2 = nn.UpsamplingNearest2d(scale_factor=2.0)
        self.conv4 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.conv6 = nn.Conv2d(64, 3, kernel_size=3, padding=1)

        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.pool(x)

        x = self.upsample1(x)
        x = self.relu(self.conv3(x))
        x = self.upsample2(x)
        x = self.relu(self.conv4(x))
        x = self.conv6(x)
        return x

    def training_step(self, batch, batch_idx):
        print(batch_idx)
        high_res, _ = batch
        low_res = self.getLowRes(high_res)
        low_res = low_res.to(device='cuda')

        output = self(low_res)
        loss = nn.MSELoss()(output, high_res)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        high_res, _ = batch
        low_res = self.getLowRes(high_res)
        low_res = low_res.to(device='cuda')
        high_res = high_res.to(device='cuda')
        
        output = self(low_res)
        loss = nn.MSELoss()(output, high_res)
        
        ssim = StructuralSimilarityIndexMeasure(data_range=1.0).to(device='cuda')
        psnr = PeakSignalNoiseRatio().to(device='cuda')
        
        metricPsnr = psnr(output, high_res)
        metricSsim = ssim(output, high_res)
        
        self.log('val_loss', loss, sync_dist=True)
        wandb.log({"train/loss": loss})
        wandb.log({"train/psnr": metricPsnr})
        wandb.log({"train/ssim": metricSsim})

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=0.001)
        return optimizer

    def getLowRes(self, high_res):
        img_array = high_res.cpu().numpy()
        img_array = np.transpose(img_array, (0, 2, 3, 1))
        resized_images = []
        for img in img_array:
            resized_img = cv2.resize(img, (400, 400))  # Resize to (64, 64)
            resized_images.append(resized_img)
        resized_images = np.array(resized_images)
        resized_tensor = torch.from_numpy(resized_images)
        # Transpose back to tensor shape (32, 3, 64, 64)
        return resized_tensor.permute(0, 3, 1, 2)

In [ ]:
import torch.utils.data

torch.set_float32_matmul_precision('high')
from lightning.pytorch.loggers import WandbLogger
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader
gc.collect()
# Example usage
high_res_folder = 'Dataset'
transform = transforms.Compose([
    transforms.Resize((800, 800)),
    transforms.ToTensor()
])

high_res_dataset = ImageFolder(high_res_folder, transform=transform)

generator1 = torch.Generator().manual_seed(21)

train_set, val_set, other = torch.utils.data.random_split(high_res_dataset, [0.6, 0.1, 0.3], generator1)
train_loader = DataLoader(train_set, batch_size = 12, shuffle=False,  num_workers=3)
val_loader = DataLoader(val_set, batch_size=12, shuffle=False, num_workers=3)

del high_res_dataset
del other


model = Upscaler((400,400))
wandb_logger = WandbLogger(project="Upscaler", log_model=True)

# Создание объекта Trainer и обучение модели
trainer = pl.Trainer(max_epochs=10, accelerator="gpu", devices=2, strategy="ddp_notebook", logger=wandb_logger)
trainer.fit(model, train_loader, val_loader)


INFO: GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO: ----------------------------------------------------------------------------------------------------
distributed_backend=nccl
All distributed processes registered. Starting with 2 processes
----------------------------------------------------------------------------------------------------

/opt/conda/lib/python3.10/site-packages/lightning/pytorch/loggers/wandb.py:396: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
/opt/conda/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:653: Checkpoint directory ./Upscaler/bkhl3ce1/checkpoints exists and is not empty.


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

0
0
1
1
22

33

4
4
55

6
6
77

88

9
9
1010

11
11
1212

1313

14
14
1515

16
16
1717

1818

19
19
20
20
2121

2222

2323

2424

25
25
2626

2727

28
28
2929

3030

3131

32
32
33
33
3434

3535

36
36
3737

3838

3939

40
40
41
41
4242

4343

4444

4545

4646

4747

4848

4949

5050

5151

5252

5353

54
54
5555

5656

57
57
5858

5959

6060

6161

6262

6363

6464

6565

6666

6767

6868

6969

7070

7171

7272

7373

7474

75
75
7676

7777

7878

7979

8080

8181

8282

83
83
84
84
8585

8686

8787

8888

8989

9090

9191

9292

9393

9494

9595

9696

9797

9898

9999

100
100
101
101
102
102
103103

104104

105105

106106

107107

108108

109109

110110

111111

112112

113
113
114114

115115

116116

117117

118118

119
119
120
120
121121

122122

123123

124124

125125

126126

127127

128128

129129

130130

131131

132132

133133

134134

135135

136136

137137

138138

139139

140140

141141

142
142
143143

144144

145145

146146

147147

148148

149149

150150

151151

1521

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

0
0
1
1
2
2
33

44

55

66

77

88

99

1010

1111

1212

1313

1414

1515

1616

1717

1818

1919

2020

2121

2222

2323

2424

2525

2626

27
27
28
28
2929

3030

3131

3232

3333

34
34
3535

3636

37
37
3838

3939

4040

41
41
4242

4343

4444

4545

4646

4747

4848

4949

5050

5151

5252

5353

5454

5555

5656

5757

5858

5959

6060

6161

6262

6363

6464

6565

66
66
67
67
6868

6969

7070

7171

72
72
7373

7474

7575

7676

7777

7878

79
79
8080

8181

8282

83
83
8484

8585

86
86
87
87
8888

8989

9090

9191

9292

9393

9494

9595

9696

9797

9898

9999

100100

101101

102102

103103

104104

105105

106
106
107107

108108

109109

110110

111111

112112

113113

114114

115
115
116116

117117

118118

119119

120120

121121

122122

123123

124124

125125

126126

127127

128128

129129

130130

131131

132132

133
133
134134

135135

136136

137137

138138

139139

140140

141141

142142

143143

144144

145145

146
146
147
147
148148

149
149
150150

151151

1521

Validation: |          | 0/? [00:00<?, ?it/s]

0
0
1
1
2
2
33

4
4
55

66

77

88

99

1010

1111

12
12
1313

1414

1515

1616

1717

1818

19
19
2020

2121

2222

2323

2424

2525

2626

2727

2828

2929

3030

3131

3232

3333

3434

3535

3636

3737

3838

39
39
40
40
4141

4242

4343

4444

4545

4646

4747

4848

4949

5050

5151

5252

5353

54
54
5555

5656

5757

5858

5959

6060

6161

6262

6363

6464

6565

6666

6767

6868

69
69
7070

7171

7272

7373

7474

7575

7676

7777

7878

79
79
8080

8181

8282

8383

8484

8585

8686

8787

8888

8989

9090

91
91
9292

9393

9494

9595

9696

9797

98
98
9999

100100

101101

102102

103103

104104

105
105
106106

107107

108108

109109

110110

111
111
112112

113113

114114

115115

116116

117117

118
118
119119

120120

121121

122122

123
123
124
124
125125

126126

127127

128128

129129

130130

131131

132132

133133

134
134
135135

136136

137137

138
138
139139

140
140
141141

142142

143143

144144

145145

146146

147147

148148

149149

150150

151151

1521

Validation: |          | 0/? [00:00<?, ?it/s]